## COSC522: Final Project - Catifier
### Cameron Adkins, Purnachandra Anirudh Gajjala, Gabriel Abeyie

In [1]:
# Numpy.
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view

# Need plots.
import matplotlib.pyplot as plt

# Pandas.
import pandas as pd

# Machine learning toolkit.
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import *
from sklearn.preprocessing import Normalizer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, accuracy_score

# Scipy for fft's and the like.
import scipy as sc
import scipy.io.wavfile as wavfile
from scipy import signal
from scipy.fftpack import fft, fftfreq
from scipy import stats

# Seaborn for plots.
import seaborn as sns

# IPython for basic visual output types.
import IPython

# Imbalanced learn for rebalancing.
from imblearn.over_sampling import SMOTE

# Standard Python libs.
import os
import glob
import csv
import xml.etree.ElementTree as et
from dataclasses import dataclass

# Pillow.
import PIL as pil
from PIL import ImageDraw

# Tensorflow
#import tensorflow as tf
#from tensorflow.keras.preprocessing.image import ImageDataGenerator
#from tensorflow.keras.datasets import fashion_mnist
#from tensorflow.keras.layers import Dense
#from tensorflow.keras.optimizers import Adam
#from tensorflow.keras.layers import Conv2D, Flatten, Dense, AveragePooling2D, GlobalAveragePooling2D,Dropout
#from tensorflow.keras.applications.resnet import ResNet50
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.preprocessing.image import ImageDataGenerator

# For images
from skimage.color import rgb2gray
import cv2
from scipy import ndimage

In [2]:
# Utility functions.

def image_load(filename):
    loader = pil.Image.open(filename);
    ret = loader.copy();
    loader.close();
    
    return ret;

def image_resize(image, new_width):
    new_height = int(new_width * (image.height / image.width));
    
    return image.resize((new_width, new_height), pil.Image.Resampling.LANCZOS);

def trimap_to_mask(trimap, include_border = True):
    trimap_data = trimap.getdata();
    
    mask_data = np.zeros((trimap.height, trimap.width), dtype=np.uint8);
    
    for x in range(0, trimap.width):
        for y in range(0, trimap.height):
            idx = x + (y * trimap.width);
            tri = trimap_data[idx];
            
            if (tri == 1 or (include_border == True and tri == 3)):
                mask_data[y, x] = 255;
            else:
                mask_data[y, x] = 0;
                
    mask = pil.Image.fromarray(mask_data);
    
    return mask;

In [3]:
# Class definitions.

@dataclass
class Point:
    x: int;
    y: int;

@dataclass
class BoundingBox:
    ll: Point;
    lr: Point;
    ul: Point;
    ur: Point;

class CatBreedSample:
    def __init__(self, label, image_file, mask_file = None, bb_file = None):
        self.label = label;
        
        self.image_file = image_file;
        self.mask_file  = mask_file;
        self.bb_file    = bb_file;
        
        # Load the image
        self.image = image_load(self.image_file);

        # Composite if a mask is available.
        if (self.mask_file):
            self.mask = trimap_to_mask(image_load(self.mask_file), False);
            
            background = pil.Image.new("RGB", self.mask.size, 0);
            self.masked_image = pil.Image.composite(self.image, background, self.mask);
        else:
            self.mask = None;
            self.masked_image = None;
            
        # Get a bounding box.
        if (self.bb_file):
            tree = et.parse(self.bb_file);
            root = tree.getroot();

            xmin = int(root.findall("./object/bndbox/xmin")[0].text);
            xmax = int(root.findall("./object/bndbox/xmax")[0].text);
            ymin = int(root.findall("./object/bndbox/ymin")[0].text);
            ymax = int(root.findall("./object/bndbox/ymax")[0].text);

            self.bb = BoundingBox(0, 0, 0, 0);
            
            self.bb.ll = Point(xmin, ymin);
            self.bb.lr = Point(xmax, ymin);
            self.bb.ul = Point(xmin, ymax);
            self.bb.ur = Point(xmax, ymax);
            
            if (self.masked_image):
                self.bounded_image = self.masked_image.crop((xmin, ymin, xmax, ymax));
            else:
                self.bounded_image = self.image.crop((xmin, ymin, xmax, ymax));

        else:
            self.bb = None;
            self.bounded_image = None;

        self.image = image_resize(self.image, 256);
        self.masked_image = image_resize(self.masked_image, 256);
        self.bounded_image = image_resize(self.bounded_image, 256);

    def display(self):
        print("Image:", self.image_file);
        print("Label:", self.label);
        
        display(self.image);
        
        if (self.mask):
            display(self.masked_image);
            
        if (self.bb):
            display(self.bounded_image);
            
    def features(self):
        fv = [];
        
        # Determine eye positions.
        
        # Determine ear positions.
        # We just estimate by starting at corners of the region segmentation and working
        # our way towards the center. The first non-black pixel is our x/y.
        region_segmentation = self.segmentation_regionbased();
        
        #Left
        
        left_ear = (0, 0);

        #print(len(region_segmentation), len(region_segmentation[0]))
        #print(seg_data[0], seg_data[1], seg_data[2], seg_data[3]);

        for y in range(0, int(len(region_segmentation) / 2)):
            for x in range(0, int(len(region_segmentation[0]) / 2)):
                #print(region_segmentation[y, x], end = " ")
                if (region_segmentation[x, y] != 0):
                    #print(x, y, "=", region_segmentation[y, x]);
                    left_ear = (y, x);
                    break;
                    
            if (left_ear != (0, 0)):
                break;

        #print(left_ear);
                
        #Right
    
        right_ear = (0, 0);

        for y in range(0, int(len(region_segmentation) / 2)):
            for x in reversed(range(int(len(region_segmentation[0]) / 2), int(len(region_segmentation[0])))):
                #print(region_segmentation[y, x], end = " ")
                if (region_segmentation[y, x] != 0):
                    #region_segmentation[y, x] = 0.5;
                    right_ear = (x, y);
                    break;
                    
            if (right_ear != (0, 0)):
                break;

        #print(right_ear);
    
        #plt.imshow(region_segmentation, cmap='gray');
        
        #draw = ImageDraw.Draw(self.bounded_image);
        #ellipse = ((left_ear[0] - 2, left_ear[1] - 2, left_ear[0] + 2, left_ear[1] + 2));
        #draw.ellipse(ellipse, fill = "red", outline = "red");
        #ellipse = ((right_ear[0] - 2, right_ear[1] - 2, right_ear[0] + 2, right_ear[1] + 2));
        #draw.ellipse(ellipse, fill = "red", outline = "red");
        
        fv.append(right_ear[0] - left_ear[0]);
        fv.append(right_ear[1] - left_ear[1]);
        
        # Bin the colors that make up this image.
        #region_segmentation = self.segmentation_colorclustering();
        #unique, counts = np.unique(region_segmentation, return_counts = True);
        
        #unique_colors = dict(zip(unique, counts));
        
        #counts = counts[1:];
        #_, counts = np.unique(counts, return_counts = True);
        
        cv2image = np.array(self.masked_image);
        
        channels = cv2.split(cv2image);
        
        for i in range(0, 3):
            hist = cv2.calcHist([channels[i]], [0], None, [5], [0, 255]);
            for val in hist:
                fv.append(int(val));
            #print(hist);
        #plt.figure();
        #plt.plot(hist);
        #plt.xlim([0, 256])
        #plt.ylim([0, 1200])
        #region_segmentation
        
        #print(unique);

        return fv;
    
    def segmentation_regionbased(self):
        gray = rgb2gray(np.array(self.bounded_image));
        #plt.imshow(gray, cmap = 'gray')
        
        gray_r = gray.reshape(gray.shape[0]*gray.shape[1])
        
        for i in range(gray_r.shape[0]):
            
            if gray_r[i] > gray_r.mean():
                
                gray_r[i] = 1

            else:
                
                gray_r[i] = 0

        gray = gray_r.reshape(gray.shape[0],gray.shape[1])
        #plt.figure();
        #plt.imshow(gray, cmap='gray')
        
        return gray;
        
        # The darker region (black) represents the background and the brighter (white) region is the foreground. We can define multiple thresholds as well to detect multiple objects:

        # gray_r = gray.reshape(gray.shape[0]*gray.shape[1])

        #for i in range(gray_r.shape[0]):
            #if gray_r[i] > gray_r.mean():
                #gray_r[i] = 3
            #elif gray_r[i] > 0.5:
                #gray_r[i] = 2
            #elif gray_r[i] > 0.25:
                #gray_r[i] = 1
            #else:
                #gray_r[i] = 0

        #gray = gray_r.reshape(gray.shape[0],gray.shape[1])
        #plt.imshow(gray, cmap='gray')
            
    def segmentation_edgebased(self):
        gray = rgb2gray(np.array(self.bounded_image));
                
        #plt.figure();
        #plt.imshow(gray, cmap='gray');

        # defining the sobel filters

        # [
        #  [ 1  2  1]
        #  [ 0  0  0]
        #  [-1 -2 -1]
        # ] 
        # is a kernel for detecting horizontal edges

        # [
        #  [-1  0  1]
        #  [-2  0  2]
        #  [-1  0  1]
        # ] 
        # is a kernel for detecting vertical edges

        sobel_horizontal = np.array([np.array([1, 2, 1]), np.array([0, 0, 0]), np.array([-1, -2, -1])])
        print('Kernel for detecting horizontal edges:\n', sobel_horizontal)

        sobel_vertical = np.array([np.array([-1, 0, 1]), np.array([-2, 0, 2]), np.array([-1, 0, 1])])
        print('Kernel for detecting vertical edges:\n', sobel_vertical)

        out_h = ndimage.convolve(gray, sobel_horizontal, mode='reflect')
        out_v = ndimage.convolve(gray, sobel_vertical, mode='reflect')

        # here mode determines how the input array is extended when the filter overlaps a border.

        plt.figure();
        plt.imshow(out_h, cmap='gray')
        plt.imshow(out_v, cmap='gray')

        # Here, we are able to identify the horizontal as well as the vertical edges. There is one more type of filter that can detect both horizontal and vertical edges at the same time. This is called the laplace operator:

        # [
        #  [1  1  1]
        #  [1 -8  1]
        #  [1  1  1]
        # ]

        kernel_laplace = np.array([np.array([1, 1, 1]), np.array([1, -8, 1]), np.array([1, 1, 1])])
        print("Laplacian kernel:\n", kernel_laplace)
        
        out_l = ndimage.convolve(gray, kernel_laplace, mode='reflect')
        plt.figure();
        plt.imshow(out_l, cmap='gray')

    def segmentation_colorclustering(self):
        # According to wikipedia the R, G, and B components of an object’s color in a digital image are all correlated with the amount of light hitting the object, 
        # and therefore with each other, image descriptions in terms of those components make object discrimination difficult. 
        # Descriptions in terms of hue/lightness/chroma or hue/lightness/saturation are often more relevant. So, we need to convert our image from RGB Colours Space to HSV to work ahead.

        cv2img = np.array(self.masked_image);

        vectorized = np.float32(cv2img.reshape((-1,3)))
        vectorized.shape

        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 20, 1.0)

        K = 8
        attempts=10
        ret, label, center = cv2.kmeans(vectorized, K, None, criteria, attempts, cv2.KMEANS_PP_CENTERS)

        center = np.uint8(center)
        res = center[label.flatten()]
        result_image = res.reshape((cv2img.shape))

        # result_image is the output result. Lets see how the image looks after k-means clustering

        #figure_size = 15
        #plt.figure(figsize=(figure_size, figure_size))
        #plt.subplot(2,3,1),plt.imshow(cv2img)
        #plt.title('Original Image'), plt.xticks([]), plt.yticks([])
        #plt.subplot(2,3,2),plt.imshow(result_image)
        #plt.title('Segmented Image when K = %i' % K), plt.xticks([]), plt.yticks([])
        #plt.show()
        
        return result_image;
    
    def haarcascade_classifier(self):
        cv2img  = np.array(self.image);
        cv2gray = cv2.cvtColor(cv2img, cv2.COLOR_RGB2GRAY);
        
        cascade = cv2.CascadeClassifier("pretrained/haarcascade_frontalcatface.xml");
        
        bounding_rects = cascade.detectMultiScale(cv2gray, scaleFactor = 1.3, minNeighbors = 1, minSize = (25, 25));
        
        print(bounding_rects);
        
        for (x, y, w, h) in bounding_rects:
            cv2.rectangle(cv2gray, (x, y), (x + w, y + h), (0, 0, 255), 2);
            
        plt.figure();
        plt.imshow(cv2gray, cmap='gray');
        
    def contours(self):
        #region_segmentation = self.segmentation_regionbased();
        region_segmentation = np.array(self.bounded_image);
        region_segmentation = cv2.cvtColor(region_segmentation, cv2.COLOR_RGB2GRAY)
        
        
        ret, threshold = cv2.threshold(region_segmentation, 16, 255, cv2.THRESH_BINARY);
        image, contours, hierarchy = cv2.findContours(threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE);

        
        cv2.drawContours(threshold, contours, -1, (0, 255, 0), 3);
        
        approx = cv2.approxPolyDP(contours[1], 0.01 * cv2.arcLength(contours[1], True), True)
        
        print(approx);
        
        plt.figure();
        plt.imshow(threshold);
        
        #i = 0;

        #for contour in contours:
        #    # here we are ignoring first counter because 
        #    # findcontour function detects whole image as shape
        #    if i == 0:
        #        i = 1
        #        continue
        #
        #    # cv2.approxPloyDP() function to approximate the shape
        #    #approx = cv2.approxPolyDP(contour, 0.01 * cv2.arcLength(contour, True), True)
        #    
        #    print(contour)
        #    
        #    # using drawContours() function
        #    cv2.drawContours(region_segmentation, [contour], 0, (0, 0, 255), 5)
        
        #plt.figure();
        #plt.imshow(region_segmentation);

In [4]:
#test_sample = CatBreedSample(
#    "Abyssinian",
#    "/home/cva/catifier/training_data/Abyssinian/Abyssinian_115.jpg",
#    "/home/cva/catifier/training_data/Abyssinian/Abyssinian_115_mask.png",
#    "/home/cva/catifier/training_data/Abyssinian/Abyssinian_115_bb.xml"
#)

#test_sample.segmentation_regionbased();
#test_sample.segmentation_edgebased();
#test_sample.segmentation_colorclustering();
#test_sample.haarcascade_classifier();
#test_sample.contours();

#fv = test_sample.features();

#test_sample.display();

In [5]:
# Load everything.
def load_samples(samples_dir):
    class_dirs = glob.glob(samples_dir + "/*")

    samples = [];

    for class_dir in class_dirs:
        label = os.path.basename(class_dir);
        class_dir_glob = glob.glob(class_dir + "/*.jpg");
    
        print("Reading files for label '" + label + "'");
    
        for sample_image in class_dir_glob:
            basename  = os.path.splitext(sample_image)[0];
            mask_file = basename + "_mask.png";
            bb_file   = basename + "_bb.xml";
            
            if (not os.path.exists(mask_file)):
                mask_file = None;
                
            if (not os.path.exists(bb_file)):
                continue;
                #bb_file = None;
            
            print(
                len(samples), 
                ":", 
                os.path.basename(sample_image), 
                "\t(mask:", 
                (mask_file != None), 
                "| bb:", 
                (bb_file != None), 
                ")"
            );
            
            sample = CatBreedSample(label, sample_image, mask_file, bb_file);
            samples.append(sample);

    return samples;

PWD = os.getcwd();
TRAINING_DATA = PWD + "/training_data";
samples = load_samples(TRAINING_DATA);

Reading files for label 'Abyssinian'
0 : Abyssinian_117.jpg 	(mask: True | bb: True )
1 : Abyssinian_103.jpg 	(mask: True | bb: True )
2 : Abyssinian_155.jpg 	(mask: True | bb: True )
3 : Abyssinian_143.jpg 	(mask: True | bb: True )
4 : Abyssinian_150.jpg 	(mask: True | bb: True )
5 : Abyssinian_165.jpg 	(mask: True | bb: True )
6 : Abyssinian_16.jpg 	(mask: True | bb: True )
7 : Abyssinian_118.jpg 	(mask: True | bb: True )
8 : Abyssinian_183.jpg 	(mask: True | bb: True )
9 : Abyssinian_123.jpg 	(mask: True | bb: True )
10 : Abyssinian_159.jpg 	(mask: True | bb: True )
11 : Abyssinian_158.jpg 	(mask: True | bb: True )
12 : Abyssinian_142.jpg 	(mask: True | bb: True )
13 : Abyssinian_114.jpg 	(mask: True | bb: True )
14 : Abyssinian_125.jpg 	(mask: True | bb: True )
15 : Abyssinian_140.jpg 	(mask: True | bb: True )
16 : Abyssinian_138.jpg 	(mask: True | bb: True )
17 : Abyssinian_181.jpg 	(mask: True | bb: True )
18 : Abyssinian_127.jpg 	(mask: True | bb: True )
19 : Abyssinian_161.jpg 

169 : Sphynx_194.jpg 	(mask: True | bb: True )
170 : Sphynx_14.jpg 	(mask: True | bb: True )
171 : Sphynx_184.jpg 	(mask: True | bb: True )
172 : Sphynx_182.jpg 	(mask: True | bb: True )
173 : Sphynx_102.jpg 	(mask: True | bb: True )
174 : Sphynx_146.jpg 	(mask: True | bb: True )
175 : Sphynx_11.jpg 	(mask: True | bb: True )
176 : Sphynx_199.jpg 	(mask: True | bb: True )
177 : Sphynx_205.jpg 	(mask: True | bb: True )
178 : Sphynx_157.jpg 	(mask: True | bb: True )
179 : Sphynx_19.jpg 	(mask: True | bb: True )
180 : Sphynx_103.jpg 	(mask: True | bb: True )
181 : Sphynx_129.jpg 	(mask: True | bb: True )
182 : Sphynx_105.jpg 	(mask: True | bb: True )
183 : Sphynx_169.jpg 	(mask: True | bb: True )
184 : Sphynx_134.jpg 	(mask: True | bb: True )
185 : Sphynx_168.jpg 	(mask: True | bb: True )
186 : Sphynx_181.jpg 	(mask: True | bb: True )
187 : Sphynx_15.jpg 	(mask: True | bb: True )
188 : Sphynx_136.jpg 	(mask: True | bb: True )
189 : Sphynx_200.jpg 	(mask: True | bb: True )
190 : Sphynx_158.

331 : Persian_162.jpg 	(mask: True | bb: True )
332 : Persian_160.jpg 	(mask: True | bb: True )
333 : Persian_134.jpg 	(mask: True | bb: True )
334 : Persian_101.jpg 	(mask: True | bb: True )
335 : Persian_143.jpg 	(mask: True | bb: True )
336 : Persian_172.jpg 	(mask: True | bb: True )
337 : Persian_19.jpg 	(mask: True | bb: True )
338 : Persian_120.jpg 	(mask: True | bb: True )
339 : Persian_166.jpg 	(mask: True | bb: True )
340 : Persian_135.jpg 	(mask: True | bb: True )
341 : Persian_16.jpg 	(mask: True | bb: True )
342 : Persian_183.jpg 	(mask: True | bb: True )
343 : Persian_100.jpg 	(mask: True | bb: True )
344 : Persian_189.jpg 	(mask: True | bb: True )
345 : Persian_137.jpg 	(mask: True | bb: True )
346 : Persian_206.jpg 	(mask: True | bb: True )
347 : Persian_180.jpg 	(mask: True | bb: True )
348 : Persian_190.jpg 	(mask: True | bb: True )
349 : Persian_159.jpg 	(mask: True | bb: True )
350 : Persian_191.jpg 	(mask: True | bb: True )
351 : Persian_193.jpg 	(mask: True | bb: T

501 : Birman_154.jpg 	(mask: True | bb: True )
502 : Birman_122.jpg 	(mask: True | bb: True )
503 : Birman_157.jpg 	(mask: True | bb: True )
504 : Birman_177.jpg 	(mask: True | bb: True )
505 : Birman_131.jpg 	(mask: True | bb: True )
506 : Birman_160.jpg 	(mask: True | bb: True )
507 : Birman_10.jpg 	(mask: True | bb: True )
508 : Birman_143.jpg 	(mask: True | bb: True )
509 : Birman_17.jpg 	(mask: True | bb: True )
510 : Birman_119.jpg 	(mask: True | bb: True )
511 : Birman_171.jpg 	(mask: True | bb: True )
512 : Birman_173.jpg 	(mask: True | bb: True )
513 : Birman_129.jpg 	(mask: True | bb: True )
514 : Birman_168.jpg 	(mask: True | bb: True )
515 : Birman_184.jpg 	(mask: True | bb: True )
516 : Birman_137.jpg 	(mask: True | bb: True )
517 : Birman_180.jpg 	(mask: True | bb: True )
518 : Birman_181.jpg 	(mask: True | bb: True )
519 : Birman_176.jpg 	(mask: True | bb: True )
520 : Birman_175.jpg 	(mask: True | bb: True )
521 : Birman_124.jpg 	(mask: True | bb: True )
522 : Birman_16

675 : Bombay_14.jpg 	(mask: True | bb: True )
676 : Bombay_113.jpg 	(mask: True | bb: True )
677 : Bombay_188.jpg 	(mask: True | bb: True )
678 : Bombay_153.jpg 	(mask: True | bb: True )
679 : Bombay_120.jpg 	(mask: True | bb: True )
680 : Bombay_135.jpg 	(mask: True | bb: True )
681 : Bombay_110.jpg 	(mask: True | bb: True )
682 : Bombay_15.jpg 	(mask: True | bb: True )
683 : Bombay_194.jpg 	(mask: True | bb: True )
684 : Bombay_100.jpg 	(mask: True | bb: True )
685 : Bombay_107.jpg 	(mask: True | bb: True )
686 : Bombay_119.jpg 	(mask: True | bb: True )
687 : Bombay_164.jpg 	(mask: True | bb: True )
688 : Bombay_19.jpg 	(mask: True | bb: True )
689 : Bombay_181.jpg 	(mask: True | bb: True )
690 : Bombay_115.jpg 	(mask: True | bb: True )
Reading files for label 'Russian_Blue'
691 : Russian_Blue_198.jpg 	(mask: True | bb: True )
692 : Russian_Blue_197.jpg 	(mask: True | bb: True )
693 : Russian_Blue_125.jpg 	(mask: True | bb: True )
694 : Russian_Blue_10.jpg 	(mask: True | bb: True )
6

837 : Bengal_185.jpg 	(mask: True | bb: True )
838 : Bengal_17.jpg 	(mask: True | bb: True )
839 : Bengal_148.jpg 	(mask: True | bb: True )
840 : Bengal_121.jpg 	(mask: True | bb: True )
841 : Bengal_109.jpg 	(mask: True | bb: True )
842 : Bengal_113.jpg 	(mask: True | bb: True )
843 : Bengal_151.jpg 	(mask: True | bb: True )
844 : Bengal_183.jpg 	(mask: True | bb: True )
845 : Bengal_136.jpg 	(mask: True | bb: True )
846 : Bengal_128.jpg 	(mask: True | bb: True )
847 : Bengal_171.jpg 	(mask: True | bb: True )
848 : Bengal_117.jpg 	(mask: True | bb: True )
849 : Bengal_134.jpg 	(mask: True | bb: True )
850 : Bengal_16.jpg 	(mask: True | bb: True )
851 : Bengal_167.jpg 	(mask: True | bb: True )
852 : Bengal_129.jpg 	(mask: True | bb: True )
853 : Bengal_112.jpg 	(mask: True | bb: True )
854 : Bengal_173.jpg 	(mask: True | bb: True )
855 : Bengal_14.jpg 	(mask: True | bb: True )
856 : Bengal_127.jpg 	(mask: True | bb: True )
857 : Bengal_154.jpg 	(mask: True | bb: True )
858 : Bengal_152

1003 : Ragdoll_101.jpg 	(mask: True | bb: True )
1004 : Ragdoll_163.jpg 	(mask: True | bb: True )
1005 : Ragdoll_18.jpg 	(mask: True | bb: True )
1006 : Ragdoll_135.jpg 	(mask: True | bb: True )
1007 : Ragdoll_136.jpg 	(mask: True | bb: True )
1008 : Ragdoll_137.jpg 	(mask: True | bb: True )
1009 : Ragdoll_131.jpg 	(mask: True | bb: True )
1010 : Ragdoll_156.jpg 	(mask: True | bb: True )
1011 : Ragdoll_103.jpg 	(mask: True | bb: True )
1012 : Ragdoll_171.jpg 	(mask: True | bb: True )
1013 : Ragdoll_176.jpg 	(mask: True | bb: True )
1014 : Ragdoll_115.jpg 	(mask: True | bb: True )
1015 : Ragdoll_125.jpg 	(mask: True | bb: True )
1016 : Ragdoll_160.jpg 	(mask: True | bb: True )
1017 : Ragdoll_172.jpg 	(mask: True | bb: True )
1018 : Ragdoll_195.jpg 	(mask: True | bb: True )
1019 : Ragdoll_182.jpg 	(mask: True | bb: True )
1020 : Ragdoll_194.jpg 	(mask: True | bb: True )
1021 : Ragdoll_134.jpg 	(mask: True | bb: True )
1022 : Ragdoll_175.jpg 	(mask: True | bb: True )
1023 : Ragdoll_193.jp

1157 : British_Shorthair_141.jpg 	(mask: True | bb: True )
1158 : British_Shorthair_153.jpg 	(mask: True | bb: True )
1159 : British_Shorthair_130.jpg 	(mask: True | bb: True )
1160 : British_Shorthair_113.jpg 	(mask: True | bb: True )
1161 : British_Shorthair_139.jpg 	(mask: True | bb: True )
1162 : British_Shorthair_105.jpg 	(mask: True | bb: True )
1163 : British_Shorthair_149.jpg 	(mask: True | bb: True )
1164 : British_Shorthair_103.jpg 	(mask: True | bb: True )
1165 : British_Shorthair_186.jpg 	(mask: True | bb: True )
1166 : British_Shorthair_178.jpg 	(mask: True | bb: True )
1167 : British_Shorthair_102.jpg 	(mask: True | bb: True )
1168 : British_Shorthair_189.jpg 	(mask: True | bb: True )
1169 : British_Shorthair_15.jpg 	(mask: True | bb: True )
1170 : British_Shorthair_119.jpg 	(mask: True | bb: True )
1171 : British_Shorthair_111.jpg 	(mask: True | bb: True )
1172 : British_Shorthair_173.jpg 	(mask: True | bb: True )
1173 : British_Shorthair_177.jpg 	(mask: True | bb: True 

In [6]:
#sample_id = 120;
#samples[sample_id].display();
#samples[sample_id].segmentation_regionbased();
#samples[sample_id].segmentation_edgebased();
#samples[sample_id].segmentation_colorclustering();

In [8]:
# Try the model.
fv = [];
labels = [];

for sample in samples:
    print("Extract features from:", sample.image_file);
    fv.append(sample.features());
    labels.append(sample.label);
    
# Rebalance.
oversampler = SMOTE();
(fv, labels) = oversampler.fit_resample(fv, labels);

# Scale.
scaler = RobustScaler();
fv = scaler.fit_transform(fv);
    
# Split the data.
x_train, x_test, y_train, y_test = train_test_split(fv, labels, test_size = 0.30, random_state = 64);

# Train.
dt = RandomForestClassifier();
dt.fit(x_train, y_train);

# Testing the model.
cv_scores = cross_val_score(dt, x_train, y_train, cv = 10);

print('Average Cross Validation Score from Training:', cv_scores.mean(), sep = '\n', end = '\n\n\n');

y_pred = dt.predict(x_test);
cm = confusion_matrix(y_test, y_pred);
cr = classification_report(y_test, y_pred);

print('Confusion Matrix:', cm, sep = '\n', end = '\n\n\n');
print('Missing classifications (if any):', set(y_test) - set(y_pred));
print('Test Statistics:', cr, sep = '\n', end = '\n\n\n');
print('Testing Accuracy:', accuracy_score(y_test, y_pred));

Extract features from: /home/cva/catifier/training_data/Abyssinian/Abyssinian_117.jpg
Extract features from: /home/cva/catifier/training_data/Abyssinian/Abyssinian_103.jpg
Extract features from: /home/cva/catifier/training_data/Abyssinian/Abyssinian_155.jpg
Extract features from: /home/cva/catifier/training_data/Abyssinian/Abyssinian_143.jpg
Extract features from: /home/cva/catifier/training_data/Abyssinian/Abyssinian_150.jpg
Extract features from: /home/cva/catifier/training_data/Abyssinian/Abyssinian_165.jpg
Extract features from: /home/cva/catifier/training_data/Abyssinian/Abyssinian_16.jpg
Extract features from: /home/cva/catifier/training_data/Abyssinian/Abyssinian_118.jpg
Extract features from: /home/cva/catifier/training_data/Abyssinian/Abyssinian_183.jpg
Extract features from: /home/cva/catifier/training_data/Abyssinian/Abyssinian_123.jpg
Extract features from: /home/cva/catifier/training_data/Abyssinian/Abyssinian_159.jpg
Extract features from: /home/cva/catifier/training_data

Extract features from: /home/cva/catifier/training_data/Abyssinian/Abyssinian_108.jpg
Extract features from: /home/cva/catifier/training_data/Abyssinian/Abyssinian_101.jpg
Extract features from: /home/cva/catifier/training_data/Abyssinian/Abyssinian_113.jpg
Extract features from: /home/cva/catifier/training_data/Sphynx/Sphynx_203.jpg
Extract features from: /home/cva/catifier/training_data/Sphynx/Sphynx_195.jpg
Extract features from: /home/cva/catifier/training_data/Sphynx/Sphynx_116.jpg
Extract features from: /home/cva/catifier/training_data/Sphynx/Sphynx_112.jpg
Extract features from: /home/cva/catifier/training_data/Sphynx/Sphynx_138.jpg
Extract features from: /home/cva/catifier/training_data/Sphynx/Sphynx_153.jpg
Extract features from: /home/cva/catifier/training_data/Sphynx/Sphynx_202.jpg
Extract features from: /home/cva/catifier/training_data/Sphynx/Sphynx_185.jpg
Extract features from: /home/cva/catifier/training_data/Sphynx/Sphynx_186.jpg
Extract features from: /home/cva/catifie

Extract features from: /home/cva/catifier/training_data/Egyptian_Mau/Egyptian_Mau_11.jpg
Extract features from: /home/cva/catifier/training_data/Egyptian_Mau/Egyptian_Mau_136.jpg
Extract features from: /home/cva/catifier/training_data/Egyptian_Mau/Egyptian_Mau_100.jpg
Extract features from: /home/cva/catifier/training_data/Egyptian_Mau/Egyptian_Mau_149.jpg
Extract features from: /home/cva/catifier/training_data/Egyptian_Mau/Egyptian_Mau_130.jpg
Extract features from: /home/cva/catifier/training_data/Egyptian_Mau/Egyptian_Mau_107.jpg
Extract features from: /home/cva/catifier/training_data/Egyptian_Mau/Egyptian_Mau_138.jpg
Extract features from: /home/cva/catifier/training_data/Egyptian_Mau/Egyptian_Mau_168.jpg
Extract features from: /home/cva/catifier/training_data/Egyptian_Mau/Egyptian_Mau_18.jpg
Extract features from: /home/cva/catifier/training_data/Egyptian_Mau/Egyptian_Mau_15.jpg
Extract features from: /home/cva/catifier/training_data/Egyptian_Mau/Egyptian_Mau_147.jpg
Extract featu

Extract features from: /home/cva/catifier/training_data/Persian/Persian_20.jpg
Extract features from: /home/cva/catifier/training_data/Persian/Persian_136.jpg
Extract features from: /home/cva/catifier/training_data/Persian/Persian_173.jpg
Extract features from: /home/cva/catifier/training_data/Persian/Persian_194.jpg
Extract features from: /home/cva/catifier/training_data/Persian/Persian_175.jpg
Extract features from: /home/cva/catifier/training_data/Persian/Persian_15.jpg
Extract features from: /home/cva/catifier/training_data/Persian/Persian_108.jpg
Extract features from: /home/cva/catifier/training_data/Persian/Persian_132.jpg
Extract features from: /home/cva/catifier/training_data/Persian/Persian_107.jpg
Extract features from: /home/cva/catifier/training_data/Persian/Persian_186.jpg
Extract features from: /home/cva/catifier/training_data/Persian/Persian_158.jpg
Extract features from: /home/cva/catifier/training_data/Persian/Persian_185.jpg
Extract features from: /home/cva/catifier/

Extract features from: /home/cva/catifier/training_data/Siamese/Siamese_202.jpg
Extract features from: /home/cva/catifier/training_data/Siamese/Siamese_182.jpg
Extract features from: /home/cva/catifier/training_data/Siamese/Siamese_127.jpg
Extract features from: /home/cva/catifier/training_data/Siamese/Siamese_129.jpg
Extract features from: /home/cva/catifier/training_data/Siamese/Siamese_145.jpg
Extract features from: /home/cva/catifier/training_data/Siamese/Siamese_184.jpg
Extract features from: /home/cva/catifier/training_data/Siamese/Siamese_158.jpg
Extract features from: /home/cva/catifier/training_data/Siamese/Siamese_169.jpg
Extract features from: /home/cva/catifier/training_data/Siamese/Siamese_116.jpg
Extract features from: /home/cva/catifier/training_data/Siamese/Siamese_172.jpg
Extract features from: /home/cva/catifier/training_data/Siamese/Siamese_150.jpg
Extract features from: /home/cva/catifier/training_data/Siamese/Siamese_175.jpg
Extract features from: /home/cva/catifie

...

Extract features from: /home/cva/catifier/training_data/Bengal/Bengal_104.jpg
Extract features from: /home/cva/catifier/training_data/Bengal/Bengal_123.jpg
Extract features from: /home/cva/catifier/training_data/Bengal/Bengal_126.jpg
Extract features from: /home/cva/catifier/training_data/Bengal/Bengal_177.jpg
Extract features from: /home/cva/catifier/training_data/Bengal/Bengal_174.jpg
Extract features from: /home/cva/catifier/training_data/Bengal/Bengal_130.jpg
Extract features from: /home/cva/catifier/training_data/Bengal/Bengal_155.jpg
Extract features from: /home/cva/catifier/training_data/Bengal/Bengal_108.jpg
Extract features from: /home/cva/catifier/training_data/Bengal/Bengal_143.jpg
Extract features from: /home/cva/catifier/training_data/Bengal/Bengal_142.jpg
Extract features from: /home/cva/catifier/training_data/Bengal/Bengal_124.jpg
Extract features from: /home/cva/catifier/training_data/Bengal/Bengal_180.jpg
Extract features from: /home/cva/catifier/training_data/Bengal/B

Extract features from: /home/cva/catifier/training_data/Maine_Coon/Maine_Coon_108.jpg
Extract features from: /home/cva/catifier/training_data/Maine_Coon/Maine_Coon_13.jpg
Extract features from: /home/cva/catifier/training_data/Maine_Coon/Maine_Coon_197.jpg
Extract features from: /home/cva/catifier/training_data/Maine_Coon/Maine_Coon_124.jpg
Extract features from: /home/cva/catifier/training_data/Maine_Coon/Maine_Coon_141.jpg
Extract features from: /home/cva/catifier/training_data/Maine_Coon/Maine_Coon_205.jpg
Extract features from: /home/cva/catifier/training_data/Maine_Coon/Maine_Coon_17.jpg
Extract features from: /home/cva/catifier/training_data/Maine_Coon/Maine_Coon_180.jpg
Extract features from: /home/cva/catifier/training_data/Maine_Coon/Maine_Coon_189.jpg
Extract features from: /home/cva/catifier/training_data/Maine_Coon/Maine_Coon_169.jpg
Extract features from: /home/cva/catifier/training_data/Maine_Coon/Maine_Coon_186.jpg
Extract features from: /home/cva/catifier/training_data/

Extract features from: /home/cva/catifier/training_data/Ragdoll/Ragdoll_130.jpg
Extract features from: /home/cva/catifier/training_data/Ragdoll/Ragdoll_101.jpg
Extract features from: /home/cva/catifier/training_data/Ragdoll/Ragdoll_163.jpg
Extract features from: /home/cva/catifier/training_data/Ragdoll/Ragdoll_18.jpg
Extract features from: /home/cva/catifier/training_data/Ragdoll/Ragdoll_135.jpg
Extract features from: /home/cva/catifier/training_data/Ragdoll/Ragdoll_136.jpg
Extract features from: /home/cva/catifier/training_data/Ragdoll/Ragdoll_137.jpg
Extract features from: /home/cva/catifier/training_data/Ragdoll/Ragdoll_131.jpg
Extract features from: /home/cva/catifier/training_data/Ragdoll/Ragdoll_156.jpg
Extract features from: /home/cva/catifier/training_data/Ragdoll/Ragdoll_103.jpg
Extract features from: /home/cva/catifier/training_data/Ragdoll/Ragdoll_171.jpg
Extract features from: /home/cva/catifier/training_data/Ragdoll/Ragdoll_176.jpg
Extract features from: /home/cva/catifier

Extract features from: /home/cva/catifier/training_data/British_Shorthair/British_Shorthair_134.jpg
Extract features from: /home/cva/catifier/training_data/British_Shorthair/British_Shorthair_108.jpg
Extract features from: /home/cva/catifier/training_data/British_Shorthair/British_Shorthair_167.jpg
Extract features from: /home/cva/catifier/training_data/British_Shorthair/British_Shorthair_127.jpg
Extract features from: /home/cva/catifier/training_data/British_Shorthair/British_Shorthair_204.jpg
Extract features from: /home/cva/catifier/training_data/British_Shorthair/British_Shorthair_155.jpg
Extract features from: /home/cva/catifier/training_data/British_Shorthair/British_Shorthair_145.jpg
Extract features from: /home/cva/catifier/training_data/British_Shorthair/British_Shorthair_168.jpg
Extract features from: /home/cva/catifier/training_data/British_Shorthair/British_Shorthair_180.jpg
Extract features from: /home/cva/catifier/training_data/British_Shorthair/British_Shorthair_16.jpg
E

Extract features from: /home/cva/catifier/training_data/British_Shorthair/British_Shorthair_122.jpg
Extract features from: /home/cva/catifier/training_data/British_Shorthair/British_Shorthair_135.jpg
Extract features from: /home/cva/catifier/training_data/British_Shorthair/British_Shorthair_136.jpg
Extract features from: /home/cva/catifier/training_data/British_Shorthair/British_Shorthair_110.jpg
Average Cross Validation Score from Training:
0.3547619047619047


Confusion Matrix:
[[10  6  1  1  1  1  3  1  0  2  1  3]
 [ 4 10  2  3  0  0  5  2  0  0  1  0]
 [ 0  1  6  1  2  4  1  2  6  1  5  1]
 [ 0  0  0 24  1  0  1  0  0  1  0  0]
 [ 0  2  1  1 15  1  5  3  3  4  0  0]
 [ 1  1  2  0  0  7  0  0  0  5  7  2]
 [ 5  7  2  1  0  0  4  4  2  1  3  1]
 [ 4  1  0  0  1  0  1  9  4  2  1  3]
 [ 3  0  4  0  1  4  0  5  6  0  5  3]
 [ 1  1  0  2  8  4  1  1  1 11  0  0]
 [ 0  5  7  1  0  3  7  1  3  0  5  3]
 [10  1  3  0  2  3  2  1  1  1  2  8]]


Missing classifications (if any): set()
Test